# The PSHA Calculation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bakerjw/shra-jupyter/blob/main/notebooks/06%20PSHA%20Calculation.ipynb)

In [7]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

### Import GMM_Functions from local src directory

In [8]:
import sys
sys.path.append('../src')
import GMM_Functions as f

## Example from Section 6.3.3

### Specify a gound-motion intensity measure (IM) and values of interest

In [9]:
T = 1 # SA period of interest
x = np.logspace(np.log10(0.001), np.log10(2), 100)  # IM values to consider

### Specify properties to predict ground-motion intensity

In [10]:
# Define combine rupture and site parameters into a dictionary
R = 10 # Horizontal distance from top of rupture measured perpendicular to fault strike (km)
Rrup = R # Closest distance (km) to the rupture plane
Rjb = R # Joyner-Boore distance (km); closest distance (km) to surface projection of rupture plane
rup = {
    'Fault_Type': 1,  # 1 is strike slip
    'Vs30': 500, # Shear wave velocity averaged over top 30 m in m/s
    'R': 10,
    'Ztor': 0, # Rupture depth
    'delta': 90, # The angle between the fault and a horizontal plane (90 = vertical fault)
    'rupLambda': 0, # The direction a hanging wall block moves during rupture (0 means left lateral motion of the hanging wall relative to the footwall)
    'Z10': 999, # Basin depth (km); depth from the groundsurface to the 1 km/s shear wave horizon
    'Fhw': 0, # Hanging wall flag (=1 for hanging wall, = 0 for foot wall)
    'FVS30': 0, # 0 for Vs30 is inferred from geology, 1 for measured Vs30,
    'region': 1, # 0 for global (incl. Taiwan), 1 for California, 2 for Japan, 3 for China, 4 for Italy , 5 for Turkey
    }

## 3. Compute the locations, characteristics, and occurrence rates of all rupture scenarios capable of producing damaging ground motions

### Gutenberg-Richter Distribution

Annual rate of exceedance for the bounded Gutenberg-Richter distribution is:

$$ λ(M≥m) = λ(M≥m_{min})(1-F_M(m)) $$

In [11]:
b = 1
m_min = 5
m_max = 8
min_rate = 0.05
delta_m = 0.2 # magnitude discretization interval

In [5]:
# Make a discretized list of potential earthquake magnitudes
m_range = list(np.arange(m_min,m_max+delta_m, delta_m))

# compute mid-points of the discretized intervals, m_vals 
m_vals_list = []
for index, elem in enumerate(m_range):
    if index == len(m_range)-1:
      break
    mid = (m_range[index] + m_range[index+1])/2
    m_vals_list.append(mid)
m_vals = np.array(m_vals_list)
print(m_vals)

[5.1 5.3 5.5 5.7 5.9 6.1 6.3 6.5 6.7 6.9 7.1 7.3 7.5 7.7 7.9]


In [ ]:
beta = np.log(10) * b

lambda_m = []
for m in m_range:
  fm = (1 - np.exp(-beta*(m - m_min)))/(1-np.exp(-beta*(m_max-m_min)))
  lambda_m.append(min_rate * (1-fm))
lambda_m.pop()

# Rate of exceedance
print(lambda_m)

[np.float64(0.05), np.float64(0.03152939662063028), np.float64(0.01987523376143628), np.float64(0.012521954111659545), np.float64(0.007882348310616167), np.float64(0.004954954954954943), np.float64(0.0031078946170179757), np.float64(0.001942478331098574), np.float64(0.0012071503661209072), np.float64(0.0007431897860165738), np.float64(0.00045045045045044585), np.float64(0.0002657444166567491), np.float64(0.0001492027880648117), np.float64(7.566999156704558e-05), np.float64(2.9273933556611677e-05)]


## 4) Predict the resulting distribution of ground-motion intensity as a function of the site characteristics and each rupture scenario’s properties (Chapters 4 and 5) & 5) Consider all possible ruptures, and uncertainty in resulting ground-motion intensity (Figure 6.1 and Equation 6.1).

In [ ]:
# Compute PSHA, with rupture rates for each M precomputed

# Created by Jack Baker
# Translated by AI

# INPUTS
# lambda_m      exceedance rate of EQs for each M
# m_vals        values of M corresponding to lambda_m
# x             IM values of interest
# x_example     example IM value for table
# rup           data structure with rupture and site parameters
# gmpe_flag      =1 for BJF97, =2 for CY14

def fn_psha_given_m_lambda(lambda_m, m_vals, t, x, x_example, rup, gmpe_flag):
    # Compute PSHA, with rupture rates for each M precomputed

    # Find occurrence rates from exceedance rates
    lambda_occur = np.concatenate((np.diff(lambda_m) * -1, [lambda_m[-1]]))

    # Initialize dictionaries for results
    lambda_result = {'x': [], 'example': []}
    disagg = {'all': np.zeros((len(x), len(m_vals)))}

    # p(exceeding each x threshold value | M)
    for j in range(len(x)):
        p_given_m = np.zeros(len(m_vals))
        for i in range(len(m_vals)):
            sa, sigma = f.gmm_eval(t, m_vals[i], rup, gmpe_flag)
            p_given_m[i] = 1 - norm.cdf(np.log(x[j]), np.log(sa), sigma)

        lambda_result['x'].append(np.sum(lambda_occur * p_given_m))
        disagg['all'][j, :] = (lambda_occur * p_given_m) / lambda_result['x'][j]

    # calcs for example IM case
    p_ex = np.zeros(len(m_vals))
    for i in range(len(m_vals)):
        sa, sigma = f.gmm_eval(t, m_vals[i], rup, gmpe_flag)
        p_ex[i] = 1 - norm.cdf(np.log(x_example), np.log(sa), sigma)

    example_output = np.column_stack((np.arange(1, len(m_vals)+1), m_vals, lambda_occur, p_ex, lambda_occur * p_ex))
    lambda_result['example'].append(np.sum(lambda_occur * p_ex))
    disagg['example'] = (lambda_occur * p_ex) / lambda_result['example']
    disagg['m_bar'] = np.sum(m_vals * disagg['example'])

    # disagg conditional on occurrence for example IM case
    x_inc = x_example * 1.02  # do computations at an increment on x
    p_inc = np.zeros(len(m_vals))
    for i in range(len(m_vals)):
        sa, sigma = f.gmm_eval(t, m_vals[i], rup, gmpe_flag)
        p_inc[i] = 1 - norm.cdf(np.log(x_inc), np.log(sa), sigma)
    lambda_inc = np.sum(lambda_occur * p_inc)
    disagg['equal'] = ((lambda_occur * p_ex) - (lambda_occur * p_inc)) / (lambda_result['example'] - lambda_inc)
    disagg['equal_m_bar'] = np.sum(m_vals * disagg['equal'])

    # disaggs with epsilon
    delta_eps = 1  # final binning
    eps_vals = np.arange(-3, 3 + delta_eps, delta_eps)  # epsilon bins

    delta_eps_fine = 0.1  # initial finer binning
    eps_vals_fine = np.arange(-3.5, 3.5 + delta_eps_fine, delta_eps_fine)  # midpoints of bins
    p_eps = norm.pdf(eps_vals_fine) * delta_eps_fine  # estimate PDF using a PMF with discrete epsilon increments
    lambda_m_and_eps = np.outer(lambda_occur, p_eps)  # rate of events with a given magnitude and epsilon

    ind = np.zeros((len(m_vals), len(eps_vals_fine)), dtype=bool)
    for i in range(len(m_vals)):
        sa, sigma = f.gmm_eval(t, m_vals[i], rup, gmpe_flag)
        ind[i, :] = (np.log(sa) + eps_vals_fine * sigma > np.log(x_example))  # indicator that the M/epsilon value causes IM > x_example
    exceed_rates_fine = ind * lambda_m_and_eps  # rates of given M/epsilon values exceeding IM
    lambda_exceed = np.sum(exceed_rates_fine)  # this is close to lambda.example, but may differ by a few percent due to epsilon discretization

    # compute mean epsilon
    eps_deagg = np.sum(exceed_rates_fine, axis=0) / np.sum(exceed_rates_fine)
    disagg['eps_bar'] = np.sum(eps_vals_fine * eps_deagg)

    # aggregate results to coarser epsilon bins
    exceed_rates = np.zeros((len(m_vals), len(eps_vals)))
    for j in range(len(eps_vals)):
        idx = (eps_vals_fine >= (eps_vals[j] - delta_eps / 2)) & (eps_vals_fine < (eps_vals[j] + delta_eps / 2))
        exceed_rates[:, j] = np.sum(exceed_rates_fine[:, idx], axis=1)

    disagg['eps_vals'] = eps_vals  # return bin midpoints
    disagg['m_eps'] = exceed_rates / lambda_exceed  # magnitude and epsilon disaggregation
    disagg['eps'] = np.sum(exceed_rates, axis=0) / lambda_exceed  # epsilon disaggregation

    disagg['equal_m_bar'] = np.sum(m_vals * disagg['equal'])

    return lambda_result, example_output, disagg

In [ ]:
x_example = 0.2  # example values for table
gmpe_flag = 1
# Compute the PSHA given M and lambda
lambda_, example_output, disagg = fn_psha_given_m_lambda(lambda_m, m_vals, T, x, x_example, rup, gmpe_flag)

x_example2 = 0.5  # output results for a second threshold
lambda2, example_output2, disagg2 = fn_psha_given_m_lambda(lambda_m, m_vals, T, x, x_example2, rup, gmpe_flag)

## PLOTS

In [ ]:
# Plot set up
plt.close('all')
plt.rcParams.update({'font.size': 12})
color_spec = [
    [56 / 255, 95 / 255, 150 / 255],
    [207 / 255, 89 / 255, 33 / 255],
    [158 / 255, 184 / 255, 219 / 255],
]

# Plotting parameters
figure_axis_limits = [0.05, max(x), 1 / 0.99e-5, 1e-1]
figure_x_tick_vals = [0.05, 0.1, 0.5, 1, 2]

im_label = 'SA(1 s)'
axis_label = 'Spectral Acceleration, SA(1 s) [g]'

In [ ]:
# Hazard curve plot
fig, ax = plt.subplots()
ax.loglog(x, lambda_['x'], '-', linewidth=2, color=color_spec[0])
ax.plot(x_example, lambda_['example'], 'o', color=color_spec[0])
ax.plot(x_example2, lambda2['example'], 'o', color=color_spec[0])

# Annotate text results for example cases
text1 = f"$\lambda$({im_label} > {x_example} g) = {lambda_['example'][0]:.3g}"
text2 = f"$\lambda$({im_label} > {x_example2} g) = {lambda2['example'][0]:.3g}"
ax.text(x_example * 1.1, lambda_['example'][0] * 1.2, text1, fontsize=8)
ax.text(x_example2 * 1.1, lambda2['example'][0] * 1.2, text2, fontsize=8)

plt.xlabel(axis_label)
plt.ylabel('Annual rate of exceedance, $\lambda$')
ax.set_xscale('log')
plt.xlim([0.05, 2])
plt.xticks([0.05, 0.1, 0.5, 1,  2], ['0.05', '0.1', '0.5', '1', '2'])


plt.show()

In [ ]:
# Output a subset of the hazard curve for use in a table
im_small = np.array([1e-3] + list(np.arange(0.1, 1.1, 0.1)))
rates_small = np.exp(interp1d(np.log(x), np.log(lambda_['x']), kind='linear', fill_value='extrapolate')(np.log(im_small)))
haz_table = np.column_stack((im_small, rates_small))

In [ ]:
# Disaggregation plot
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.bar(m_vals, disagg['example'], width=0.2, color=color_spec[2], edgecolor='black')
plt.plot([disagg['m_bar'], disagg['m_bar']], [0, 1], ':k', linewidth=2)
plt.xlabel('Magnitude, M')
plt.ylabel(f'P(m | {im_label} > {x_example} g)')
plt.axis([5, 8, 0, 0.2])

plt.subplot(1, 2, 2)
plt.bar(m_vals, disagg2['example'], width=0.2, color=color_spec[2], edgecolor='black')
plt.plot([disagg2['m_bar'], disagg2['m_bar']], [0, 1], ':k', linewidth=2)
plt.xlabel('Magnitude, M')
plt.ylabel(f'P(m | {im_label} > {x_example2} g)')
# plt.axis([5, 8, 0, 0.2])
plt.ylim([0, 0.2])
plt.xlim([4,8])

plt.tight_layout()

In [ ]:
m_bar = [disagg['m_bar'], disagg2['m_bar']]

# Tabulate output
disagg_table = np.column_stack((m_vals, disagg['example'], disagg2['example']))

# Metrics to evaluate calculations and figure
# Interpolate IM with given rate
rate_targ = 1 / 1000
im_targ = np.exp(interp1d(np.log(rates_small), np.log(im_small), kind='linear', fill_value='extrapolate')(np.log(rate_targ)))

# Manual log interpolation
ln_im_manual = ((np.log(0.2) - np.log(0.3)) * (np.log(1E-3) - np.log(6.81E-4))) / (np.log(2.7E-3) - np.log(6.81E-4)) + np.log(0.3)
im_manual = np.exp(ln_im_manual)

# Hazard curves slope
im_slope = [0.2, 0.3]
rate_slope = np.exp(interp1d(np.log(x), np.log(lambda_['x']), kind='linear', fill_value='extrapolate')(np.log(im_slope)))
k_est = - (np.log(rate_slope[0]) - np.log(rate_slope[1])) / (np.log(im_slope[0]) - np.log(im_slope[1]))
k0_est = rate_slope[0] / np.exp(-k_est * np.log(im_slope[0]))
lambda_power_law = k0_est * np.exp(-k_est * np.log(x))

# Hazard curve derivative
d_lambda = -np.diff(np.concatenate((rates_small, [0])))

In [ ]:
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.bar(im_small + 0.05, d_lambda, width=0.10, color=color_spec[2], edgecolor='black')
plt.axis([0, 1, 0, 0.05])
# plt.text(-0.1, -0.07, '(a)', transform=plt.gca().transAxes, verticalalignment='center')
plt.xlabel(axis_label)
plt.ylabel('$\Delta \lambda_i$')

plt.subplot(1, 2, 2)
x_fine = np.arange(0.01, 1.01, 0.01)
lambda_fine = np.exp(interp1d(np.log(x), np.log(lambda_['x']), kind='linear', fill_value='extrapolate')(np.log(x_fine)))
d_lambda_fine = -np.diff(np.concatenate((lambda_fine, [0])))
plt.bar(x_fine + 0.005, d_lambda_fine, width=0.008, color=color_spec[2],edgecolor='black')
plt.axis([0, 1, 0, 0.008])
plt.xlabel(axis_label)
plt.ylabel('$\Delta \lambda_i$')

plt.tight_layout()

In [ ]:
# Summary plot
plt.figure()
h1 = plt.loglog(x, lambda_['x'], '-', linewidth=2, color=color_spec[0], label='Original hazard curve')
plt.plot(im_targ, rate_targ, 'ok')
h2 = plt.plot(x, lambda_power_law, '-', linewidth=2, color=color_spec[2], label='Fitted power-law hazard curve')
plt.plot([0.01, im_targ, im_targ], [rate_targ, rate_targ, 1e-10], ':k', linewidth=1)
plt.text(im_targ * 1.05, rate_targ * 1.5, f'$\lambda$({im_label} > {im_targ:.3g} g) = {rate_targ:.3g}')
plt.text(0.01 * 1.05, lambda_m[0] * 1.25, f'$\Sigma_i \lambda(rup_i)$ = {lambda_m[0]}')

plt.legend(handles=[h1[0], h2[0]])
plt.xlabel(axis_label)
plt.ylabel('Annual rate of exceedance, $\lambda$')
plt.xlim([0.01,2])
plt.ylim([10e-6,10e-2])
plt.xticks([0.05, 0.1, 0.5, 1, 2], ['0.05', '0.1', '0.5', '1', '2'])
plt.show()